In [1]:
#!pip install playsound
#!pip install gtts

In [2]:
import cv2
import face_recognition
import os
import numpy as np
import time
from playsound import playsound
from gtts import gTTS

In [3]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{}:{}".format(h, m, s)

In [4]:
def text_speech(text):
    tts = gTTS(text, lang='ml')
    tts.save('dummy.mp3')
    sound=playsound('dummy.mp3')
    media_path=os.path.join(cur_path,'dummy.mp3') 
    os.remove(media_path)
    return sound

In [5]:
cur_path=r"C:\My Directory\Knowledge\CVDL\OpenCV\face_rec\meeting"

In [6]:
myList = os.listdir(cur_path)
print(myList)

['Andrej_Karpathy.png', 'Elon_Musk.jpg', 'Sreejesh.jpg']


In [7]:
Images = []
Names = []
for cl in myList:
    try:
        img_path =os.path.join(cur_path, cl)
        curImg = cv2.imread(f'{img_path}')
        Images.append(curImg)
        Names.append(os.path.splitext(cl)[0])
    except:
        print('could not process')
print(Names)
print(len(Images))

['Andrej_Karpathy', 'Elon_Musk', 'Sreejesh']
3


In [8]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
    


In [9]:
encodeListKnown = findEncodings(Images)
print(len(encodeListKnown))

3


In [10]:
video_capture = cv2.VideoCapture()
while True:
    # Capture frame-by-frame
    start = time.time()
    success, img = video_capture.read()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        matchIndex = np.argmin(faceDis)
        if faceDis[matchIndex]< 0.50:
            name = Names[matchIndex].upper()
            text1= 'Namaskaram'+ name
            text_speech(text1)
        else: 
            name = 'Unknown'
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            text2= "shradikkuka: Oru aparichitan vannitundu"
            text_speech(text2)
            cv2.imshow('Webcam',img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            break
        break
        # Display the resulting frame

error: OpenCV(4.6.0) C:\b\abs_74oeeuevib\croots\recipe\opencv-suite_1664548340488\work\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()
elapsed = time.time()-start
print (f'user time: {hms_string(elapsed)}')